We are going to explore the idea of expression trees and how they relate to
our tree structures, namely `TreeNode`, and to evaluate the expression trees
by rewriting the nodes in post-order traversal.

First, let's define our expression tree.

In [1]:
from treekit.treenode import TreeNode
expr = TreeNode({
    "value": "+",
    "type": "op",
    "children": [
        {
            "value": "max",
            "type": "op",
            "children": [
                {
                    "value": "+", 
                    "type": "op",
                    "children": [{"type": "var", "value": "x"},
                                 {"type": "const", "value": 1}]
                },
                { "type": "const", "value": 0 }
            ]
        },
        {"type": "op",
         "value": "+",
         "children": [
             {"type": "op", "value": "*",
              "children": [{"type": "var", "value": "x"},
                           {"type": "var", "value": "y"}]},
             {"type": "const", "value": 3}, 
             {"type": "var", "value": "y"}],
        }
    ]
})

Here is what that looks like in a more convenient form.

In [2]:
from treekit.tree_viz import TreeViz
print(TreeViz.text(expr, node_name=lambda n: n.value))

+
├── max
│   ├── +
│   │   ├── x
│   │   └── 1
│   └── 0
└── +
    ├── *
    │   ├── x
    │   └── y
    ├── 3
    └── y



As a tree structure, `TreeNode` implements an interface that permits
tree traversal algorithms like depth_first pre-order and post-order traversals.

We are going to implement a simple post-order traversal algorithm to permit
computation of the exression tree we defined earlier, `expr`. We see that
it contains 3 operator types, `+`, `*`, and `max`, numbers, and variables.

We will provide a **closure** over all of these types so that when we evaluate
expression in post-order, all of types are defined for the operations.

In [3]:
def postorder(node, fn, ctx):
    """
    Applies function `fn` to the nodes in the tree using post-order traversal.
    :param fn: Function to apply to each node. Should accept one argument: the node.
    :return: The tree with the function `fn` applied to its nodes.
    """
    node[TreeNode.CHILDREN_KEY] = [postorder(child, fn, ctx) for child in node.children()]
    return fn(node, ctx)


The function `postorder` takes a tree node `node`, a function
`fn : Node, Context -> Node`, and a context `ctx`, and returns a rewritten
tree.

At each node, `postorder` recursively calls `fn` on its children before applying
`fn` to the node itself. This is the essence of post-order traversal.

Post-order is useful for problems where the children need to be processed before
the node itself. For example, **evaluating** an expression tree, where typicallyt
he value of a node can only be computed after the values of its children are known.

If we do a pre-order traversal of the tree, we apply `fn` to the node before
applying it to the children. For example, if we need to rewrite
the tree in a different form, for instance algebraic simplification, preorder
may be useful.

In what follows, we design a simple expression tree evaluator, `Eval`.

In [4]:
from functools import reduce
from copy import deepcopy

class Eval:
    """
    An evaluator for a expressions defined by operations on types, respectively
    defined by `Eval.Op`  and `Eval.Type`. The operations are a
    dictionary where the keys are the operation names and the values are
    functions that take a node and a context and return the value of the
    operation in that context.
    """

    Op = {
        '+': lambda x: sum(x),
        '*': lambda x: reduce(lambda a, b: a*b, x),
        'max': lambda x: max(x)
    }

    Type = {
        'const': lambda node, _: node['value'],
        'var': lambda node, ctx: ctx[node['value']],
        'op': lambda node, _: Eval.Op[node['value']](c['value'] for c in node.children())
    }

    def __init__(self, debug=True):
        """
        :param debug: If True, print debug information
        """
        self.debug = debug
    
    def __call__(self, expr, ctx):
        
        def _eval(node, ctx):
            node_type = Eval.Type[node['type']]
            value = Eval.Type[node['type']](node, ctx)
            result = TreeNode(type='const', value=value)
            if self.debug:
                print(f"Eval({node.get_data()} -> {result.get_data()})")
            return result

        return postorder(deepcopy(expr), _eval, ctx)


To evaluate an express tree, we need for the operations to be defined for all
of the types when we do post-order (bottom-up) traversal. We can define a
closure over all of the types, and then use that closure to evaluate the
expression tree.

We call this closure a context. Normally, the operations and other thingsar
e also defined in the closure, but for simplicity we will just define the
operations and provide closures over the variables.

In [5]:
ctx = {
    "x": 1,
    "y": 2,
    "z": 3  
}
result = Eval(debug=True)(expr, ctx)

Eval({'type': 'var', 'value': 'x'} -> {'type': 'const', 'value': 1})
Eval({'type': 'const', 'value': 1} -> {'type': 'const', 'value': 1})
Eval({'value': '+', 'type': 'op'} -> {'type': 'const', 'value': 2})
Eval({'type': 'const', 'value': 0} -> {'type': 'const', 'value': 0})
Eval({'value': 'max', 'type': 'op'} -> {'type': 'const', 'value': 2})
Eval({'type': 'var', 'value': 'x'} -> {'type': 'const', 'value': 1})
Eval({'type': 'var', 'value': 'y'} -> {'type': 'const', 'value': 2})
Eval({'type': 'op', 'value': '*'} -> {'type': 'const', 'value': 2})
Eval({'type': 'const', 'value': 3} -> {'type': 'const', 'value': 3})
Eval({'type': 'var', 'value': 'y'} -> {'type': 'const', 'value': 2})
Eval({'type': 'op', 'value': '+'} -> {'type': 'const', 'value': 7})
Eval({'value': '+', 'type': 'op'} -> {'type': 'const', 'value': 9})


In [6]:
print(result)

TreeNode({'type': 'const', 'value': 9})


We see that we get the expected result, $20$. Note that it is still a tree, but
it has been transformed into a so-called self-evaluating tree expression.
In this case, a single node (no children)

We can evaluate it again, and we see that it cannot be rewritten further. We
call this state of affairs a **normal form**. Essentially, we can think of the
tree as a program that computes a value, and the normal form is the result of
running the program.

In [7]:
assert Eval(debug=False)(result, ctx) == result

What happens when we change the context so that not every variable is defined?

In [8]:
open_ctx = {
    'x': 1,
    #'y': 2,
    'z': 3
}

try:
    Eval(debug=True)(expr, open_ctx)
except KeyError as e:
    print(f'Error: {e}')

Eval({'type': 'var', 'value': 'x'} -> {'type': 'const', 'value': 1})
Eval({'type': 'const', 'value': 1} -> {'type': 'const', 'value': 1})
Eval({'value': '+', 'type': 'op'} -> {'type': 'const', 'value': 2})
Eval({'type': 'const', 'value': 0} -> {'type': 'const', 'value': 0})
Eval({'value': 'max', 'type': 'op'} -> {'type': 'const', 'value': 2})
Eval({'type': 'var', 'value': 'x'} -> {'type': 'const', 'value': 1})
Error: 'y'


We see that we get an error. Our operations in `Eval.Op` are not defined over
variables.

We would run into a similar problem if we used pre-order traversal of post-orde
In preorder traversal, we would try to evaluate the parent node (say, an operation)
before we had evaluated its children, which would result in an error. Our
defined operations only work over numbers (type `const`), so we need to
evaluate the non-`const` expressions first in order for our operations to be
defined for them.

Postorder traversal is good for things like evaluating expressions, where you
need to evaluate the children before you can evaluate the parent.

Preorder traversal is good for things like rewriting trees from the top down,
but your rewrite rules need to be defined in terms of sub-expression trees.
So, for example, you might have a complex expression and seek to rewrite it
into a simpler form. This is an example of a **rewrite system**. A rewrite system
is a set of rules that transform expressions into other expressions. For
instance, suppose that we add a `0` to a variable `x` in the expression tree.
We know that `x + 0` is the same as `x`, so we could add a rewrite rule that maps the
sub-tree '(+ x 0)' to 'x'. We could add many rewrite rules to implement, for
instance, algebraic simplification (`simplify`), or implement a compiler
(`compile`) that translates the tree into a different form that could be
evaluated by a different set of rewrite rules. Or, the compiler could be an
optimizing compiler that rewrites the tree into a form that is more efficient
to evaluate, like replacing a multiplication by a power of two with a shift
or getting rid of no-op operations like adding zero.